In [ ]:
import os
# import plotly.express as px
# import plotly.graph_objs as go
import plotly.io as pio
import pandas as pd
import requests

# plotly setup
pio.renderers.default = 'notebook'
pd.options.plotting.backend = 'plotly'


def pwrite(fig, plt='/tmp/vis/plot.json'):
    fig = fig.update_layout(autosize=False)
    fig.write_json(plt)


def ensure_directory(dir):
    """ ensure the directory exists """
    if os.path.isfile(dir):
        dir = os.path.dirname(dir)
    if not os.path.exists(dir):
        os.makedirs(dir)

# SpaceTraders

In [ ]:
callsign = 'API_T3ST'
faction = 'COSMIC'
base_url = 'https://api.spacetraders.io/v2'
token_fp = os.path.expanduser('~/.config/st/tokens')
ensure_directory(token_fp)

In [ ]:
# registering a new agent
res = requests.post(
    os.path.join(base_url, 'register'),
    headers={'Content-Type': 'application/json'},
    data={
        'symbol': callsign,
        'faction': faction
    }
)
if res.ok:
    data = res.json()['data']
    with open(os.path.join(token_fp, callsign), 'w+') as f:
        f.write(data['token'])

In [ ]:
# reading token by agent callsign
with open(os.path.join(token_fp, callsign), 'r') as f:
    token = f.read()

In [ ]:
# get agent
res = requests.get(
    os.path.join(base_url, 'my/agent'),
    headers={'Authorization': f'Bearer {token}'}
)
res

In [ ]:
agent = res.json()['data']
agent

In [ ]:
hq = agent['headquarters']
hq_split = hq.split('-')
hq_sector = hq_split[0]
hq_system = '-'.join(hq_split[:2])

In [ ]:
# view your starting location
res = requests.get(
    os.path.join(base_url, 'systems', hq_system, 'waypoints', hq),
    headers={'Content-Type': 'application/json'},
)
res

In [ ]:
res.json()

In [ ]:
# view contracts
res = requests.get(
    os.path.join(base_url, 'my/contracts'),
    headers={'Authorization': f'Bearer {token}'}
)
contracts = res.json()['data']
contracts

In [ ]:
# accept a contract
contract = contracts[0]
res = requests.post(
    os.path.join(base_url, 'my/contracts', contract['id'], 'accept'),
    headers={'Authorization': f'Bearer {token}'}
)
res

In [ ]:
res.json()

In [ ]:
# find a shipyard
res = requests.get(
    os.path.join(base_url, 'systems', hq_system, 'waypoints'),
    params={'traits': 'SHIPYARD'},
    headers={'Authorization': f'Bearer {token}'}
)
res

In [ ]:
shipyards = res.json()['data']

In [ ]:
# view available ships
res = requests.get(
    os.path.join(
        base_url,
        'systems',
        hq_system,
        'waypoints',
        shipyards[1]['symbol'],
        'shipyard'
    ),
)
res.json()['data']

In [ ]:
# purchase a ship
res = requests.post(
    os.path.join(base_url, 'my/ships'),
    headers={
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    },
    json={
        'shipType': 'SHIP_MINING_DRONE',
        'waypointSymbol': shipyards[1]['symbol'],
    }
)
res

In [ ]:
res.json()['data']

In [ ]:
# find nearby asteroid
res = requests.get(
    os.path.join(base_url, 'systems', hq_system, 'waypoints'),
    params={'type': 'ENGINEERED_ASTEROID'},
    headers={'Authorization': f'Bearer {token}'}
)
res

In [ ]:
asteroid = res.json()['data'][0]

In [ ]:
# (view all ships)
res = requests.get(
    os.path.join(base_url, 'my/ships'),
    headers={'Authorization': f'Bearer {token}'},
)
res

In [ ]:
ships = res.json()['data']
for ship in ships:
    print(ship['symbol'], ship['frame']['symbol'], ship['mounts'])

In [ ]:
# fly to the asteroid

# orbit the ship at the current location
res = requests.post(
    os.path.join(base_url, 'my/ships', ships[2]['symbol'], 'orbit'),
    headers={'Authorization': f'Bearer {token}'},
)
res.json()['data']

In [ ]:
# navigate to the asteroid
res = requests.post(
    os.path.join(base_url, 'my/ships', ships[2]['symbol'], 'navigate'),
    headers={
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    },
    json={'waypointSymbol': asteroid['symbol']},
)
res

In [ ]:
res.json()['data']

In [ ]:
# dock the ship
res = requests.post(
    os.path.join(base_url, 'my/ships', ships[2]['symbol'], 'dock'),
    headers={'Authorization': f'Bearer {token}'},
)
res

In [ ]:
res.json()['data']

In [ ]:
# refuel the ship
res = requests.post(
    os.path.join(base_url, 'my/ships', ships[2]['symbol'], 'refuel'),
    headers={'Authorization': f'Bearer {token}'},
)
res

In [ ]:
res.json()['data']

In [ ]:
# orbit ship again
res = requests.post(
    os.path.join(base_url, 'my/ships', ships[2]['symbol'], 'orbit'),
    headers={'Authorization': f'Bearer {token}'},
)
res.json()['data']

In [ ]:
# extract ores & minerals
res = requests.post(
    os.path.join(base_url, 'my/ships', ships[2]['symbol'], 'extract'),
    headers={'Authorization': f'Bearer {token}'},
)
res.json()['data']

In [ ]:
# view market data
res = requests.get(
    os.path.join(
        base_url,
        'systems',
        hq_system,
        'waypoints',
        asteroid['symbol'],
        'market'
    ),
    headers={'Authorization': f'Bearer {token}'}
)
res.json()['data']

In [ ]:
# list ship cargo
res = requests.get(
    os.path.join(base_url, 'my/ships', ships[2]['symbol'], 'cargo'),
    headers={'Authorization': f'Bearer {token}'},
)
cargo = res.json()['data']
cargo

In [ ]:
# sell goods

# dock the ship
res = requests.post(
    os.path.join(base_url, 'my/ships', ships[2]['symbol'], 'dock'),
    headers={'Authorization': f'Bearer {token}'},
)

In [ ]:
# sell goods
res = requests.post(
    os.path.join(
        base_url,
        'my/ships',
        ships[2]['symbol'],
        'sell'
    ),
    headers={
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    },
    json={
        'symbol': 'IRON_ORE',
        'units': 100
    }
)

In [ ]:
# navigate to delivery waypoint
res = requests.get(
    os.path.join(base_url, 'my/ships', ships[2]['symbol'], 'navigate'),
    headers={
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    },
    json={'waypointSymbol': contracts[0]['deliver'][0]['destinationSymbol']},
)
res.json()['data']

In [ ]:
# deliver contract goods
res = requests.post(
    os.path.join(
        base_url,
        'my/contracts',
        contracts[0]['id'],
        'deliver'
    ),
    headers={
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    },
    json={
        'shipSymbol': ships[2]['symbol'],
        'tradeSymbol': contracts[0]['deliver'][0]['tradeSymbol'],
    },
)
res.json()['data']

In [ ]:
# fulfill contract
res = requests.post(
    os.path.join(
        base_url,
        'my/contracts',
        contracts[0]['id'],
        'fulfill'
    ),
    headers={'Authorization': f'Bearer {token}'}
)